In [2]:
%autosave 0
%matplotlib inline
import sys, os
sys.path.insert(0, os.path.expanduser('~/work/git/github/pymc-devs/pymc3'))

import pandas as pd

# スライドのための設定
from traitlets.config.manager import BaseJSONConfigManager
path = os.path.expanduser('~/.jupyter/nbconfig') # "/home/damian/miniconda3/envs/rise_latest/etc/jupyter/nbconfig"
cm = BaseJSONConfigManager(config_dir=path)
cm.update('livereveal', {
              'theme': 'sky',
              'transition': 'scroll',
              'start_slideshow_at': 'selected',
})

cm.update('livereveal', {
              'width': 1200,
              'height': 800,
})

cm.update('livereveal', {
              'scroll': True,
})

Autosave disabled


{'height': 800,
 'scroll': True,
 'start_slideshow_at': 'selected',
 'theme': 'sky',
 'transition': 'scroll',
 'width': 1200}

<br />

<div style="text-align: center;">
<font size="7"><b>PyMC3による階層ベイズ回帰</b></font>
</div>
<br />
<br />
<br />
<br />
<div style="text-align: right;">
<font size="6">吉岡琢</font>
</div>

<br />

# ベイズ推定
$$
p(\theta|D)=\frac{p(D,\theta)}{p(D)}=\frac{p(D|\theta)p(\theta)}{p(D)}
$$
* データ$D$から推定されたモデルパラメータ$\theta$の事後分布を推定する方法. 

## 階層ベイズ
$$
p(\theta_{0},\theta|D)=\frac{p(D,\theta,\theta_{0})}{p(D)}=\frac{p(D|\theta)p(\theta|\theta_{0})p(\theta_{0})}{p(D)}
$$
* 事前分布を階層化した階層事前分布を用いる方法. 


# PyMC3
* ベイズ推定を簡単に実行できるPythonライブラリです. 
* 最近 :code:`pip` でインストールできるようになりました. 
* 事後分布にしたがうモデルパラメータのサンプルが得られます. 

## 例：予測分布
$$
p(d|D)=\int p(d|\theta)p(\theta|D)d\theta\sim\frac{1}{M}\sum_{m=1}^{M}p(d|\theta^{m})
$$
* 予測分布の計算に必要なサンプル$\theta^{m}\sim p(\theta|D)$がPyMC3に実装されたサンプリングアルゴリズム（MCMC）により得られます. 

    * Metropolis, Gibbs sampling, Hamiltonian MC, NUTS
    
* 同様の方法で予測分布の平均も計算できます. 

# ここで取り上げる問題
* PyMC3の開発者Thomas Wieckiさんのブログより引用しました. 

    * http://twiecki.github.io/blog/2014/03/17/bayesian-glms-3/

# データ
![Image](http://www.fix-your-radon.com/images/how_radon_enters.jpg)
* 説明変数：家がある地域, 地下の有無
* 従属変数：家で計測されたラドンの量

## データのロード

In [4]:
data = pd.read_csv(os.path.expanduser('~/work/git/github/pymc-devs/pymc3/docs/source/data/radon.csv'))
county_names = data.county.unique()
county_idx = data['county_code'].values
n_counties = len(county_names)

In [5]:
data[['county', 'log_radon', 'floor']].head()

,county,log_radon,floor
0,AITKIN,0.832909,1.0
1,AITKIN,0.832909,0.0
2,AITKIN,1.098612,0.0
3,AITKIN,0.095310,0.0
4,ANOKA,1.163151,0.0


# 回帰モデル
$$
radon_{i,c} = \alpha_{c} + \beta_{c}\cdot floor_{i,c} + \epsilon_{c}
$$

* $i$：家のインデックス
* $c$：地域のインデックス
* $radon_{i,c}$：地域$c$の$i$番目の家（サンプル）のラドンの量
* $\alpha_{c}$：地域$c$の回帰モデルの切片
* $\beta_{c}$：地域$c$の回帰モデルの回帰係数
* $floor_{i,c}$：地域$c$の$i$番目の家に地下室があれば1, 無ければ0
* $\epsilon_{c}$：ノイズ

# モデル1：地域ごとに回帰
![Image](http://f.cl.ly/items/38020n2t2Y2b1p3t0B0e/Screen%20Shot%202013-10-10%20at%208.23.36%20AM.png)

$$
\alpha_{c}\sim{\cal N}(0,100) \\
\beta_{c}\sim{\cal N}(0,100)
$$
* 各地域のモデルパラメータは独立した事前分布を持ちます.

# モデル2：階層ベイズ回帰
<img src="http://f.cl.ly/items/1B3U223i002y3V2W3r0W/Screen%20Shot%202013-10-10%20at%208.25.05%20AM.png" width="512" align="center"/>

$$
\mu_{c}\sim{\cal N}(0, 100)\\
\sigma_{c}\sim{\rm Uniform}(0, 100)\\
\alpha_{c}\sim{\cal N}(\mu_{\alpha},\sigma_{\alpha}) \\
\beta_{c}\sim{\cal N}(\mu_{\beta},\sigma_{\beta})
$$
* 各地域のモデルパラメータは独立した事前分布を持ちます. 
* ただし事前分布のパラメータに対する事前分布は共通です. 

# モデル1のコード

In [ ]:
import pymc3 as pm

indiv_traces = {}
for county_name in county_names:
    c_data = data.ix[data.county == county_name]
    c_log_radon = c_data.log_radon
    c_floor_measure = c_data.floor.values
    
    with pm.Model() as individual_model:
        a = pm.Normal('alpha', mu=0, sd=100**2)
        b = pm.Normal('beta', mu=0, sd=100**2)
        eps = pm.Uniform('eps', lower=0, upper=100)
    
        radon_est = a + b * c_floor_measure
        radon_like = pm.Normal('radon_like', mu=radon_est, sd=eps, observed=c_log_radon)

        step = pm.NUTS()
        trace = pm.sample(2000, step=step, progressbar=False)
    
    # この変数に, p(\theta|D)にしたがうサンプルが含まれます. 
    indiv_traces[county_name] = trace

# モデル2のコード

In [ ]:
with pm.Model() as hierarchical_model:
    # Hyperpriors for group nodes
    mu_a = pm.Normal('mu_alpha', mu=0., sd=100**2)
    sigma_a = pm.Uniform('sigma_alpha', lower=0, upper=100)
    mu_b = pm.Normal('mu_beta', mu=0., sd=100**2)
    sigma_b = pm.Uniform('sigma_beta', lower=0, upper=100)
    
    a = pm.Normal('alpha', mu=mu_a, sd=sigma_a, shape=n_counties)
    b = pm.Normal('beta', mu=mu_b, sd=sigma_b, shape=n_counties)
    eps = pm.Uniform('eps', lower=0, upper=100)
    
    radon_est = a[county_idx] + b[county_idx] * data.floor.values
    radon_like = pm.Normal('radon_like', mu=radon_est, sd=eps, observed=data.log_radon)

# サンプリング結果
![Image](Unknown.png)

# サンプリング結果の評価
* 収束性：サンプリングの時系列を見てトレンドがないことを確認します. 
* Posterior predictive check：$p(\theta|D)$から得られたサンプルによる予測と観測データの平均誤差を計算する方法です. 

## 結果
* 非階層モデルの平均誤差：0.13
* 階層モデルの平均誤差：0.08

# 推定結果の違い
![Image](Unknown2.png)